In [20]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

sparqlwd = SPARQLWrapper("https://query.wikidata.org/sparql")
myid = "wd:Q22673982"

query = """
SELECT ?item ?itemLabel 
WHERE 
{
  ?item wdt:P31 wd:Q891723. # doit avoir comme nature 'public company'
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } # le label viendra de préférence dans votre langue, et autrement en anglais
}
"""

sparqlwd.setQuery(query)
sparqlwd.setReturnFormat(JSON)
results = sparqlwd.query().convert()

In [69]:
liste_public_company = []
for r in results['results']['bindings']:
    liste_public_company.append({'name': r['itemLabel']['value'], 'source': 'wikidata', 'type': 'public company'})
df_public_company = pd.DataFrame(liste_public_company)

# scanR

In [36]:
import ast

In [22]:
df_scanr = pd.read_csv('https://data.enseignementsup-recherche.gouv.fr/api/explore/v2.1/catalog/datasets/export-des-organisations-exposees-dans-scanr/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B', sep=';')

In [54]:
def parse(x):
    try:
        return ast.literal_eval(x)
    except:
        return None

In [56]:
df_entreprises_scanr = df_scanr[(df_scanr.isFrench==1) & (df_scanr.kind=='Secteur privé')]

df_entreprises_scanr['badges_p'] = df_scanr.badges.apply(lambda x:parse(x))
df_entreprises_scanr['label_p'] = df_scanr.label.apply(lambda x:parse(x))

#df_entreprises_scanr_badge = df_entreprises_scanr[df_entreprises_scanr.badges.apply(lambda x:isinstance(get_list(x), list))]

<ipython-input-56-6c503d98849c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entreprises_scanr['badges_p'] = df_scanr.badges.apply(lambda x:parse(x))
<ipython-input-56-6c503d98849c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entreprises_scanr['label_p'] = df_scanr.label.apply(lambda x:parse(x))


In [66]:
data_scanr_badges = []
for row in df_entreprises_scanr[['label_p', 'badges_p']].dropna().itertuples():
    elt = {'name': row.label_p['default'], 'source': 'scanR', 'type': ';'.join(list(set([b['code'] for b in row.badges_p])))}
    data_scanr_badges.append(elt)
df_scanr_small = pd.DataFrame(data_scanr_badges)#.type.value_counts()

In [73]:
pd.concat([df_scanr_small, df_public_company]).to_csv('liste_entreprises.csv', sep=';')